In [1]:
from sentence_transformers import CrossEncoder, InputExample
from torch.utils.data import DataLoader
import pandas as pd
import torch

# 1. Load your data
df = pd.read_csv("../data/stage2_reranker_training_data.csv")

/opt/miniconda3/envs/crossencoder/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2. Convert to sentence-transformers InputExample format
train_samples = [
    InputExample(texts=[row['query'], row['label']], label=float(row['score']))
    for _, row in df.iterrows()
]

In [15]:
# 3. Initialize CrossEncoder model
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", num_labels=1)


In [16]:
# 4. Define callback to print loss every step
def print_loss(score, step):
    print(f"Step {step} - Loss: {score:.4f}")

# 5. Train the model (fit handles batching, loss, optimizer, etc.)
model.fit(
    train_dataloader=DataLoader(train_samples, shuffle=True, batch_size=4),
    epochs=20,
    warmup_steps=100,
    show_progress_bar=False,
    callback=print_loss
)

{'train_runtime': 21.5618, 'train_samples_per_second': 40.813, 'train_steps_per_second': 10.203, 'train_loss': 0.5570533752441407, 'epoch': 20.0}


In [17]:
model.save('../models/stage2_cross_encoder_finetuned')